<!-- #######  YAY, I AM THE SOURCE EDITOR! #########-->
<p>&nbsp;</p>
<p>&nbsp;In the current notebook we will see some basic <strong>EDA</strong> for the features and will try to find the feature with the most <strong>influence</strong> on the Pawpularity of a pet. The Dasatet consists only of binary features(1 and 0)that indicates what "characteristics" appears in the image,e.x. if there is a humam or the image is blurred. Of course they are not enought for the prediction of popularity but we can discover some intresting information about the "aesthetics" that judge the beauty of a pet<img src="https://html-online.com/editor/tiny4_9_11/plugins/emoticons/img/smiley-tongue-out.gif" alt="tongue-out" /><strong><br /></strong></p>

In [ ]:


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import cv2
import glob
import warnings

warnings.filterwarnings("ignore")

In [ ]:
plt.rcParams['figure.dpi']=80
plt.rcParams['font.size']=11
plt.rcParams['font.family']='sans-serif'
plt.rcParams["figure.figsize"]=(10,10)
plt.rcParams["figure.facecolor"]="white"

In [ ]:
ima_path=r"/kaggle/input/petfinder-pawpularity-score/train"
data_copy=pd.read_csv(r"/kaggle/input/petfinder-pawpularity-score/train.csv")
data=pd.read_csv(r"/kaggle/input/petfinder-pawpularity-score/train.csv")
data.head()

In [ ]:
def read_ima(path,labels,buff=10):
    images=np.zeros(shape=(buff,320,320,3),dtype="int")
    pawpularities=[]
    ima_path=os.path.normpath(path+"/*.jpg")
    for j,image in enumerate(glob.glob(ima_path)[:buff]):
        im=cv2.imread(image)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im=cv2.resize(im,dsize=(320,320))         
        id=image.split("/")[-1].split(".")[0]
        pawpopularity=(labels[labels["Id"]==id].Pawpularity.to_numpy())
        images[j-1]=im
        pawpularities.append(pawpopularity)

        
    return images,np.array(pawpularities)
ima,pawpol=read_ima(ima_path,data[["Id","Pawpularity"]],buff=16)

In [ ]:
fig=plt.figure(figsize=(15,15))
ax=fig.subplots(4,4)
ax=ax.ravel()
for j,axis in enumerate(ax):
    axis.imshow(ima[j])
    axis.set_title("Pawpularity:" +str(pawpol[j][0]))
    axis.set_xticks([])
    axis.set_yticks([])

In [ ]:
Id=data[["Id","Pawpularity"]]
data.drop("Id",axis=1,inplace=True)


In [ ]:
sns.distplot(data.Pawpularity,color="green")

<p>The distribution of Pawpopularity is a little bit skewed at the left and have a high value at 100..</p>

In [ ]:
features=np.array(data.columns[:-1])
fig,axes=plt.subplots(2,len(features)//2,figsize=(20,20),sharex=True,sharey=True)
axes=axes.ravel()
for j,i in enumerate(data.columns[:-1]):
    sns.histplot(data=data,x="Pawpularity",hue=i,ax=axes[j],palette=["#7bba84","#eb0505"],legend=False)
    axes[j].spines['right'].set_visible(False)
    axes[j].spines['top'].set_visible(False)
    axes[j].spines['left'].set_visible(False)
    axes[j].tick_params(axis=u'both', which=u'both',length=0)
    axes[j].legend(["0","1"], title=i,edgecolor="white")


<p>As you can see there are some features that are not &quot;balanced&quot;. They contain almost one of the two values they can take(1&quot;True&quot; or 0&quot;False&quot;). Also there is not significant difference between conditional distributions <strong>P(Pawpularity|feature)</strong>,so there is no a particular feature that has big impact on the Pawpularity</p>

In [ ]:
corr=data.corr()
up_triang = np.triu(np.ones_like(corr)).astype(bool)
sns.heatmap(corr,annot=True,fmt=".2f",mask=up_triang)


In [ ]:
fig,axes=plt.subplots(2,len(features)//2,figsize=(20,20),sharey=True)
axes=axes.ravel()
for j,i in enumerate(data.columns[:-1]):
    sns.boxplot(data=data,y="Pawpularity",x=i,ax=axes[j],palette=["#7bba84","#eb0505"])
    axes[j].spines['right'].set_visible(False)
    axes[j].spines['top'].set_visible(False)
    if (j!=0 and j!=6):
        axes[j].spines['left'].set_visible(False)
        axes[j].set_ylabel("")
        axes[j].tick_params(axis=u'y', which=u'both',length=0)

In [ ]:
fig,axes=plt.subplots(nrows=2,ncols=len(features)//2,figsize=(29,20))

fig.subplots_adjust(hspace=0.15,wspace=0.3)
fig.set_facecolor("#151716")
axes=axes.ravel()
for i,feature in enumerate(features):
    t=data[feature].value_counts().reset_index()
    sns.barplot(data=t,x="index",y=feature,ax=axes[i],palette=["#7bba84","#eb0505"])
    for bar in axes[i].patches:
        axes[i].annotate(format(bar.get_height()/len(data), '.2f')+ "%",
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()+0.01), ha='center', va='center',
                   size=13, xytext=(0, 8),
                   textcoords='offset points',color="white")
    axes[i].set_ylabel("")
    lim=np.array(axes[i].get_ylim())
    axes[i].set_xlabel("")
    axes[i].tick_params(axis='both', colors='white',size=10)
    axes[i].set_ylim(lim+[-0.2,1000])
    axes[i].text(x=-.2,y=lim[1]+500,s=feature,fontsize=13,fontfamily="serif",fontweight="heavy",color="white")
    axes[i].set_facecolor("#202925")

<p>Lets count the <strong>unique one-hot values</strong> of features(e.x 10101100) and&nbsp; calculate the <strong>mean Pawpularity</strong></p>


In [ ]:
#it can be achieved with pandas.DataFrame.value_counts method in one line(data.iloc[:,:-1].value_counts()[:10].reset_index())
#Just for flex purpose the below code
vectors=[]
for i in range(len(data)):
    values=data.iloc[i].values[:-1]
    value="".join([str(num) for num in values])
    vectors.append(value)
vectors=np.array(vectors)
unique,counts=np.unique(vectors,return_counts=True)
print("There is",len(unique)," unique One-Hot encoded values of features","\n"*3)
ind=np.argsort(counts)[::-1]
ind=ind[:10]
unique,counts=unique[ind],counts[ind]
arr=np.array([[int(i) for i in value] for value in unique])
arr=np.hstack([arr,counts[:,np.newaxis]])
unique_vec=pd.DataFrame(arr,columns=np.hstack([features,"Counts"]))
info_feature=data.merge(unique_vec,how="left",right_on=list(features),left_on=list(features),indicator=True)
info_feature=info_feature[info_feature._merge=="both"].drop("_merge",axis=1)
mean_pawpularity=info_feature.groupby(by=list(features)).Pawpularity.mean().reset_index()
unique_vec=unique_vec.merge(mean_pawpularity,how="left",right_on=list(features),left_on=list(features))
unique_vec.Pawpularity=(unique_vec.Pawpularity.round(1))
unique_vec.rename({"Pawpularity":"Mean_pawpularity"},axis="columns",inplace=True)
unique_vec

Maybe the distribution of popularity depends to specific combinations of features(ex. 1010101011).A distribution plot for combinations with the most occurrences will help us...

In [ ]:
data_encoded=pd.concat([data,pd.DataFrame(vectors,columns=["encode"])],axis=1)

In [ ]:
fig=plt.figure(figsize=(20,20))
fig.subplots_adjust(hspace=0.10,wspace=0.1)
fig.set_facecolor("#151716")
ax=fig.subplots(2,5,sharey=True)
axes=ax.ravel()
for i in range(0,len(unique)):
    temp=data[data_encoded.encode==unique[i]]
    sns.distplot(temp.Pawpularity,ax=axes[i],color="#28808a")
    for j,feature in enumerate(features):
        axes[i].text(x=57,y=(0.036-j*0.0018),s=f"{feature}:{unique[i][j]}",color="#d2d8d9",fontweight="roman",bbox=dict(facecolor="red",alpha=0.5))
      
    axes[i].set_facecolor("#151716")
    axes[i].spines['right'].set_visible(False)
    axes[i].spines['top'].set_visible(False)
    axes[i].spines['left'].set_visible(False)
    axes[i].tick_params(axis='both', colors='white',size=5)
    axes[i].tick_params(axis=u'both', which=u'both',length=0)
    axes[i].set_ylabel("")
    axes[i].set_xlabel("Pawpularity",color="white")
    

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression,mutual_info_regression
x=data.iloc[:,:-1]
y=data.iloc[:,-1]
model_F=SelectKBest(f_regression,k=5)
model_F.fit(x,y)
print(f"Selected features with F-values : {features[model_F.get_support()]}")
model_Mut=SelectKBest(mutual_info_regression,k=5)
model_Mut.fit(x,y)
print("Selected features with mutual inform. :",features[model_Mut.get_support()])

In [ ]:
from sklearn.feature_selection import SelectFromModel,RFE
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.metrics import mean_squared_error
model_Ln=LinearRegression()
model_SGD=SGDRegressor()

print("-"*30,"\nBest features with Linear Regre.\n","-"*30,sep="")
from_Ln=SelectFromModel(model_Ln,threshold="0.90*mean")
from_Ln.fit(x,y)
ind=from_Ln.get_support()
for i,j in enumerate(features[ind]):
    print(f"{j} \n Coefficient:{abs(from_Ln.estimator_.coef_[ind][i])}")

print("\n "*2)
print("-"*30,"\nBest features with SGDregressor","\n","-"*30,sep="")
from_SGD=SelectFromModel(model_SGD,threshold="0.90*mean")
from_SGD.fit(x,y)
ind=from_SGD.get_support()
for i,j in enumerate(features[ind]):
    print(f"{j} \n Coefficient:{abs(from_SGD.estimator_.coef_[ind][i])}")

<p>You can notice below that Trees gives the opposite results from the linear models. They give low importance on the high-coeff features of Linear Regression,indicating that there is possibly no&nbsp; linear relation between features and target.</p>

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
fig=plt.figure(figsize=(15,10))
ax=fig.subplots()
extre=ExtraTreesRegressor()
extre.fit(x,y)
std = np.std([
    tree.feature_importances_ for tree in extre.estimators_], axis=0)
ind=np.argsort(extre.feature_importances_)[::-1]
sns.barplot(x=features,y=extre.feature_importances_,order=features[ind],ax=ax,yerr=std,palette=sns.color_palette("flare"))
ax.set_title("Extra Trees",fontweight="bold",fontsize=15,fontstyle="italic",fontfamily="cursive")
ax.set_facecolor("#edede1")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn.model_selection import GridSearchCV
fig=plt.figure(figsize=(15,10))
ax=fig.subplots()
rand=RandomForestRegressor()
rand.fit(x,y)
std = np.std([
    tree.feature_importances_ for tree in rand.estimators_], axis=0)
ind=np.argsort(rand.feature_importances_)[::-1]
sns.barplot(x=features,y=rand.feature_importances_,order=features[ind],ax=ax,yerr=std,palette=sns.color_palette("flare"))
ax.set_title("Random Forest",fontweight="bold",fontsize=15,fontstyle="italic",fontfamily="cursive")
ax.set_facecolor("#edede1")


Because features are all of them binary, we can not create new informations from them but we can try to assign each image to a cluster. In clustering we will use the extracted features from a pretrained *EfficientB5* model. 

In [ ]:
ids=[file.split("\\")[-1].split(".")[0] for file in glob.glob(ima_path+"//*.jpg")]
import cv2
image_size=(456,456)#image's size tha EfficientB5 expects
images=[]
for file in ids:
    image=cv2.imread(os.path.join(ima_path,file)+".jpg")
    image=cv2.resize(image,dsize=image_size)
    images.append(image)
    
images=np.array(images)

In [ ]:
import tensorflow as tf
Efficient_model=tf.keras.applications.EfficientNetB5()
Model=tf.keras.models.Model(Efficient_model.input,
        Efficient_model.layers[-3].output)#model.layer[-3] is a global pooling layer and returns 2042 features per sample

In [ ]:
#preprocessing images for efficient model
pre_images=tf.keras.applications.efficientnet.preprocess_input(images)
#creating embedding on batches because they dont fit in memory at once
batch_size=8
embeddings=np.zeros(shape=(len(pre_images),2048))
for i in range(len(pre_images)//batch_size):
    embedding=Model(pre_images[i*batch_size:(i+1)*batch_size])
    embeddings[i*batch_size:(i+1)*batch_size]=embedding

First,we will fit a MiniBatchKmeans algorithm without dimensionality reduction

In [ ]:
from sklearn import cluster
from sklearn.metrics import silhouette_score


inertials=[]
#silhouettes=[]
batch_size=4*826
for clusters in range(1,11):
    mini_Kmeans=cluster.MiniBatchKMeans(n_clusters=clusters,batch_size=batch_size)
    for i in range(len(pre_images)//batch_size):
            mini_Kmeans.partial_fit(embeddings[i*batch_size:(i+1)*batch_size])
    #score=silhouette_score(embeddings,mini_Kmeans.predict(embeddings))
    #silhouettes.append(score)
    inertials.append(mini_Kmeans.inertia_)
plt.plot(list(range(1,11)),inertials)

#label_kmneans=mini_Kmeans.predict(embeddings)
#extended_data=pandas.concat([data,pd.DataFrame(label_kmneans,columns=["Labels_kmeans"])],axis=1)

I will select clusters=2 and visualize the clustered images(I will visualize for 5 clusters)

In [ ]:
from sklearn import cluster
clusters=2
batch_size=4*826
mini_Kmeans=cluster.MiniBatchKMeans(n_clusters=clusters,batch_size=batch_size)
for i in range(len(pre_images)//batch_size):
        mini_Kmeans.partial_fit(embeddings[i*batch_size:(i+1)*batch_size])
            
label_kmeans=mini_Kmeans.predict(embeddings)
extended_data=pd.concat([data_copy,pd.DataFrame(label_kmeans,columns=["Labels_kmeans"])],axis=1)
extended_data["file"]=extended_data.Id.apply(lambda x:os.path.join(ima_path,x+".jpg"))


In [ ]:
fig=plt.figure(figsize=(25,20))
fig.subplots_adjust(wspace=0.2,hspace=0.1)
ax=fig.subplots(clusters,4)
for clust in range(0,clusters):#i chose clust so there is no confliction with sklearn.cluster module 
    files=extended_data[extended_data.Labels_kmeans==clust].sample(4).file.values
    for i in range(0,4):
        ima=cv2.imread(files[i],)
        ima=cv2.cvtColor(ima,cv2.COLOR_BGR2RGB)
        ima=cv2.resize(ima,dsize=(220,220))
        ax[clust,i].imshow(ima)
        ax[clust,i].set_xticks([])
        ax[clust,i].set_yticks([])

       

In [ ]:
#cluster=5
clusters=5
batch_size=4*826
mini_Kmeans=cluster.MiniBatchKMeans(n_clusters=clusters,batch_size=batch_size)
for i in range(len(pre_images)//batch_size):
        mini_Kmeans.partial_fit(embeddings[i*batch_size:(i+1)*batch_size])
            
label_kmeans=mini_Kmeans.predict(embeddings)
extended_data=pd.concat([data_copy,pd.DataFrame(label_kmeans,columns=["Labels_kmeans"])],axis=1)
extended_data["file"]=extended_data.Id.apply(lambda x:os.path.join(ima_path,x+".jpg"))
fig=plt.figure(figsize=(25,20))
fig.subplots_adjust(wspace=0.2,hspace=0.1)
ax=fig.subplots(clusters,4)
for clust in range(0,clusters):#i chose clust so there is no confliction with sklearn.cluster module 
    files=extended_data[extended_data.Labels_kmeans==clust].sample(4).file.values
    for i in range(0,4):
        ima=cv2.imread(files[i],)
        ima=cv2.cvtColor(ima,cv2.COLOR_BGR2RGB)
        ima=cv2.resize(ima,dsize=(220,220))
        ax[clust,i].imshow(ima)
        ax[clust,i].set_xticks([])
        ax[clust,i].set_yticks([])


Now we will reduce the features maps(extracted from Efficient model) to 600 and we are going to fit Kmeans again(now there is no need for online learning because they fit in memory).PCA and TruncatedSVD are similar methods so on they will give similar "results"

In [ ]:
from sklearn.decomposition import TruncatedSVD,PCA

Svd=TruncatedSVD(n_components=800)
Svd.fit(embeddings)
print("SVD explained variance:",Svd.explained_variance_ratio_.sum())
pca=PCA(n_components=800)
pca.fit(embeddings)
print("PCA explained variance:",pca.explained_variance_ratio_.sum())

In [ ]:
from sklearn.cluster import KMeans
svd_embde=Svd.transform(embeddings)
inertias=[]
for clusters in range(1,11):
    kmeans=KMeans(n_clusters=clusters)
    kmeans.fit(svd_embde)
    inertias.append(kmeans.inertia_)
plt.plot(list(range(1,11)),inertias)

It is clear that we should select cluster=2

In [ ]:
clusters=2
kmeans=KMeans(n_clusters=clusters)
kmeans.fit(svd_embde)
svd_labels=kmeans.predict(svd_embde)
extended_data=pd.concat([data_copy,pd.DataFrame(svd_labels,columns=["SVD_Labels_kmeans"])],axis=1)
extended_data["file"]=extended_data.Id.apply(lambda x:os.path.join(ima_path,x+".jpg"))


In [ ]:
fig=plt.figure(figsize=(20,15))
fig.subplots_adjust(wspace=0.2,hspace=0.1)
ax=fig.subplots(clusters,4)
for clust in range(0,clusters):
    files=extended_data[extended_data.SVD_Labels_kmeans==clust].sample(4).file.values
    for i in range(0,4):
        ima=cv2.imread(files[i],)
        ima=cv2.cvtColor(ima,cv2.COLOR_BGR2RGB)
        ima=cv2.resize(ima,dsize=(220,220))
        ax[clust,i].imshow(ima)
        ax[clust,i].set_xticks([])
        ax[clust,i].set_yticks([])


It is clear that Kmeans try to distinguish cats from dogs but does not achieve it very well. Probably a Vggnet that uses Dense layers instead of Global pooling for his latest layers would give us better results. Also experiments with clustering methods could give better results